In [1]:
import copy
%load_ext autoreload
%autoreload 2
%run Datasets.ipynb
import numpy as np
from keras import Input, Model
from keras.src.layers import Dense, BatchNormalization, GaussianNoise
from src import HammingCode
Eb_treinamento = 5.01187

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2024-06-10 20:25:37.000380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 20:25:37.000553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 20:25:37.017844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 20:25:37.085915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:25:38.843432: W tensorflow/compiler/tf2

In [2]:
def create_autoencoder(code_rate, neurons, size, training_data, validation_data):
    
    entrada = Input(shape=(size,))
    encoder1 = Dense(size, activation='relu')(entrada)
    encoder2 = Dense(neurons, activation='linear')(encoder1)
    encoder3 = BatchNormalization()(encoder2)
    
    
    Eb_training = Eb_treinamento * code_rate
    
    noise = GaussianNoise(np.sqrt(1/(2* Eb_training)))(encoder3)
    
    decoder1 = Dense(size, activation='relu')(noise)
    decoder2 = Dense(size, activation='softmax')(decoder1)
    
    autoencoder = Model(entrada, decoder2)
    autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    autoencoder.fit(
    training_data,
    training_data,
    epochs=20,
    batch_size=300,
    validation_data=(validation_data, validation_data),
    )

    # Com o modelo treinado, separa em encoder e decoder
    encoder = Model(entrada, encoder3)
    
    entrada_encoder = Input(shape=(neurons,))
    decoder_layer1 = autoencoder.layers[-2](entrada_encoder)
    decoder_layer2 = autoencoder.layers[-1](decoder_layer1)
    decoder = Model(entrada_encoder, decoder_layer2)
    
    return encoder, decoder

encoder, decoder = create_autoencoder(4/7, 7, 4, training_data, validation_data)
encoder_11, decoder_11 =  create_autoencoder(11/15, 15, 11, training_data_11, validation_data_11)

Epoch 1/20
34/34 [==============================] - 3s 13ms/step - loss: 1.6654 - accuracy: 0.2125 - val_loss: 1.3881 - val_accuracy: 0.2670
Epoch 2/20
34/34 [==============================] - 0s 6ms/step - loss: 1.4145 - accuracy: 0.3100 - val_loss: 1.3417 - val_accuracy: 0.2670
Epoch 3/20
34/34 [==============================] - 0s 6ms/step - loss: 1.2206 - accuracy: 0.4645 - val_loss: 1.2966 - val_accuracy: 0.5030
Epoch 4/20
34/34 [==============================] - 0s 5ms/step - loss: 1.0720 - accuracy: 0.5564 - val_loss: 1.2313 - val_accuracy: 0.7540
Epoch 5/20
34/34 [==============================] - 0s 5ms/step - loss: 0.9353 - accuracy: 0.6584 - val_loss: 1.1426 - val_accuracy: 0.7540
Epoch 6/20
34/34 [==============================] - 0s 5ms/step - loss: 0.7992 - accuracy: 0.8348 - val_loss: 1.0159 - val_accuracy: 1.0000
Epoch 7/20
34/34 [==============================] - 0s 5ms/step - loss: 0.6591 - accuracy: 0.9488 - val_loss: 0.8675 - val_accuracy: 1.0000
Epoch 8/20
34/34 [=

In [3]:
from keras.src.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import Sequential

# MLP
noise_for_training = np.sqrt((1/(2*Eb_treinamento*(4/7)))) * np.random.randn(training_length, 4)    
noisy_input_for_training = training_data + noise_for_training

noise_for_validation = np.sqrt((1/(2*Eb_treinamento*(4/7)))) * np.random.randn(validation_length, 4)    

noisy_input_for_validation = validation_data + noise_for_validation
mlp = Sequential()
mlp.add(Input(shape=(4,)))
mlp.add(Dense(512, activation='relu'))
mlp.add(Dense(256, activation='relu'))
mlp.add(Dense(128, activation='relu'))
mlp.add(Dense(4, activation='sigmoid'))
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

mlp.fit(
    noisy_input_for_training,
    training_data,
    epochs=20,
    batch_size=300,
    validation_data=(noisy_input_for_validation, validation_data),
)

loss, accuracy = mlp.evaluate(noisy_input_for_validation, validation_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Epoch 1/20
34/34 [==============================] - 2s 15ms/step - loss: 0.3280 - accuracy: 0.8611 - val_loss: 0.1471 - val_accuracy: 0.8880
Epoch 2/20
34/34 [==============================] - 0s 10ms/step - loss: 0.1364 - accuracy: 0.8956 - val_loss: 0.1445 - val_accuracy: 0.8860
Epoch 3/20
34/34 [==============================] - 0s 11ms/step - loss: 0.1315 - accuracy: 0.8956 - val_loss: 0.1440 - val_accuracy: 0.8820
Epoch 4/20
34/34 [==============================] - 0s 9ms/step - loss: 0.1321 - accuracy: 0.8963 - val_loss: 0.1431 - val_accuracy: 0.8870
Epoch 5/20
34/34 [==============================] - 0s 9ms/step - loss: 0.1315 - accuracy: 0.8959 - val_loss: 0.1443 - val_accuracy: 0.8780
Epoch 6/20
34/34 [==============================] - 0s 10ms/step - loss: 0.1315 - accuracy: 0.8953 - val_loss: 0.1445 - val_accuracy: 0.8890
Epoch 7/20
34/34 [==============================] - 0s 9ms/step - loss: 0.1306 - accuracy: 0.8960 - val_loss: 0.1424 - val_accuracy: 0.8860
Epoch 8/20
34/34

In [ ]:
from math import erfc

Eb_dB_values = np.arange(-4, 11, 1)
ber_autoencoder = []
ber_Hamming = []
ber_teorica = []
ber_mlp = []
ber_autoencoder_11 = []
for i in range(0, len(Eb_dB_values)):
    
    
    Eb = 10**(Eb_dB_values[i]/10)
    
    testing_data_autoencoder = copy.deepcopy(testing_data)
    merged_sample = np.concatenate(testing_data_autoencoder)
    
    testing_data_hamming = copy.deepcopy(merged_sample)
    testing_data_mlp = copy.deepcopy(testing_data)
    merged_testing_data_mlp = copy.deepcopy(merged_sample)
    noise = np.sqrt((1/(2*4/7*Eb))) * np.random.randn(test_length, 7)    
    
    noise_hamming = copy.deepcopy(noise)
    noise_mlp = copy.deepcopy(noise)
    
    # codifica para 7 bits
    codificado = encoder.predict(testing_data_autoencoder)
    
    #print(codificado)
    # add ruido
    codificado_com_ruido = codificado + noise
    
    #corrige com o decoder
    corrigido = decoder.predict(codificado_com_ruido)
    corrigido_merged = np.concatenate(corrigido)
    corrigido_merged_digital = [1 if bit > 0.5 else 0 for bit in corrigido_merged]
    calculated_ber = np.sum(np.array(corrigido_merged_digital).astype(int) != merged_sample.astype(int)) / len(merged_sample)
    ber_autoencoder.append(calculated_ber)
    print('SNR ', Eb_dB_values[i], 'BER ', calculated_ber)


    ####################### Autoencoder 11
    
    testing_data_autoencoder_11 = copy.deepcopy(testing_data_11)
    merged_sample_11 = np.concatenate(testing_data_autoencoder_11)
    
    noise_11 = np.sqrt((1/(2*11/15*Eb))) * np.random.randn(test_length, 15)    
    
    # codifica para 11 bits
    codificado_11 = encoder_11.predict(testing_data_autoencoder_11)
    
    # add ruido
    codificado_com_ruido_11 = codificado_11 + noise_11
    
    #corrige com o decoder
    corrigido_11 = decoder_11.predict(codificado_com_ruido_11)
    corrigido_merged_11 = np.concatenate(corrigido_11)
    corrigido_merged_digital_11 = [1 if bit > 0.5 else 0 for bit in corrigido_merged_11]
    calculated_ber_11 = np.sum(np.array(corrigido_merged_digital_11).astype(int) != merged_sample_11.astype(int)) / len(merged_sample_11)
    ber_autoencoder_11.append(calculated_ber_11)
    print('SNR ', Eb_dB_values[i], 'BER ', calculated_ber_11)
    
    ####################### Hamming code
    # Merge sample to encode
    split_encoded_data, split_original_data, merged_encoded_data = HammingCode.encode_sample(testing_data_hamming)

    # Add noise
    merged_encoded_data = [1.0 if bit == 1 else -1.0 for bit in merged_encoded_data]
    merged_noise = np.concatenate(noise_hamming)
    merged_encoded_data = merged_encoded_data + np.array(merged_noise)
    digital_encoded_noisy = np.array([1 if x > 0 else 0.0 for x in merged_encoded_data])
    
    split_digital_encoded_noisy = np.array_split(digital_encoded_noisy, int(len(digital_encoded_noisy)/7))
    
    decoded_sample = HammingCode.decode_sample(split_digital_encoded_noisy)
    calculated_ber_Hamming = np.sum(np.array(decoded_sample).astype(int) != testing_data_hamming.astype(int)) / len(testing_data_hamming)
    ber_Hamming.append(calculated_ber_Hamming)
    
    print('SNR ', Eb_dB_values[i], 'BER ', calculated_ber_Hamming)    
    
    ####################### MLP
    noise_smaller = noise_mlp[:, :4]
    amostra_ruidosa = testing_data_mlp + noise_smaller
    
    mlp_corrigido = mlp.predict(amostra_ruidosa)
    
    mlp_corrigido_merged = np.concatenate(mlp_corrigido)
    mlp_corrigido_merged_digital = [1 if bit > 0.5 else 0 for bit in mlp_corrigido_merged]
    calculated_ber_mlp = np.sum(np.array(mlp_corrigido_merged_digital).astype(int) != merged_testing_data_mlp.astype(int)) / len(merged_testing_data_mlp)
    ber_mlp.append(calculated_ber_mlp)
    print('SNR ', Eb_dB_values[i], 'BER ', calculated_ber_mlp)

31250/31250 [==============================] - 57s 2ms/step
SNR  -4 BER  0.06977525
31250/31250 [==============================] - 67s 2ms/step
SNR  -4 BER  0.015040727272727273
SNR  -4 BER  0.26126575
31250/31250 [==============================] - 77s 2ms/step
SNR  -4 BER  0.27414425
31250/31250 [==============================] - 60s 2ms/step
SNR  -3 BER  0.0517465
31250/31250 [==============================] - 59s 2ms/step
SNR  -3 BER  0.008779
SNR  -3 BER  0.2285005
31250/31250 [==============================] - 78s 3ms/step
SNR  -3 BER  0.2608715
31250/31250 [==============================] - 59s 2ms/step
SNR  -2 BER  0.03611125
31250/31250 [==============================] - 63s 2ms/step
SNR  -2 BER  0.004475
SNR  -2 BER  0.1930565
31250/31250 [==============================] - 75s 2ms/step
SNR  -2 BER  0.24619475
31250/31250 [==============================] - 56s 2ms/step
SNR  -1 BER  0.024151
31250/31250 [==============================] - 57s 2ms/step
SNR  -1 BER  0.0019425454545

In [ ]:
Eb_values_BPSK = 10**(Eb_dB_values/10)
BPSK = [0.5 * erfc(np.sqrt(each)) for each in Eb_values_BPSK]

In [ ]:
from matplotlib import pyplot as plt

plt.semilogy(Eb_dB_values, ber_Hamming, marker='', linestyle='--', label='Codificado')
plt.semilogy(Eb_dB_values, ber_mlp, marker='', linestyle='--', label='MLP')

plt.semilogy(Eb_dB_values, ber_autoencoder, marker='', linestyle='--', label='Autoencoder')
plt.semilogy(Eb_dB_values, ber_autoencoder_11, marker='', linestyle='--', label='Autoencoder 11 bits')

plt.semilogy(Eb_dB_values, BPSK, marker='', linestyle='--', label='BPSK')
plt.xlabel('Eb/No (dB)')
plt.ylabel('BER')
plt.grid(True, which='both')
plt.legend()
plt.title('BER x Eb/No')

plt.show()
